In [1]:
from transformers import BartTokenizer, BartForConditionalGeneration
import requests
from bs4 import BeautifulSoup

d:\ProgramData\library\environments\text_mining\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = "https://edition.cnn.com/2025/11/30/asia/flooding-senyar-ditwah-indonesia-malaysia-thailand-intl-hnk"
res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(res.text, "lxml")

title = soup.find(["h1", "h2"]).get_text(strip=True)

possible_blocks = [
    "article", "main", "div.article__content", "div#main-content",
    "section", "div.story-body"
]

texts = []
for block in possible_blocks:
    body = soup.select_one(block)
    if body:
        for p in body.find_all("p"):
            texts.append(p.get_text(strip=True))
        break

content = "\n".join(texts) if texts else "Tidak menemukan paragraf isi."

print("\nTITLE:", title)
print("\nCONTENT:\n")
print(content)


TITLE: More than 1,100 killed as deadly storms cause flooding and landslides across Asia

CONTENT:

Heavy rainfall has unleashed widespread flooding and landslides across Asia, killing more than 1,100 people in a week of destruction for the region and creating treacherous search and rescue operations for hundreds more still missing.
Millions of people are dealing with intense flooding after cyclone-fueled downpours battered parts of Indonesia, Thailand and Malaysia.
Sri Lanka was struck by a separate storm, leaving some areas submerged and creating the most difficult rescue operation the country has ever seen, according to President Anura Kumara Dissanayake.
“This is the first time the entire country has been struck by such a disaster,” he said in an address to the nation, Reuters reported. His comment suggested that the breadth of devastation has surpassed the damage caused in Sri Lanka by the Asian tsunami in 2004, which killed far more people.
Across Asia, the severe weather, which

In [3]:
tokenizer = BartTokenizer.from_pretrained("brotoo/BART-NewsSummarizer")
model = BartForConditionalGeneration.from_pretrained("brotoo/BART-NewsSummarizer")

d:\ProgramData\library\environments\text_mining\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--brotoo--BART-NewsSummarizer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back t

In [4]:
inputs = tokenizer(
    [content],
    max_length=1024,
    truncation=True,
    return_tensors="pt"
)

In [5]:
summary_ids = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    num_beams=4,        # beam search
    max_length=300,
    min_length=120,
    early_stopping=True,
    no_repeat_ngram_size=3
)

In [6]:
summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [7]:
print("=== ORIGINAL ===")
print(content)
print("\n=== SUMMARY ===")
print(summary_text)

=== ORIGINAL ===
Heavy rainfall has unleashed widespread flooding and landslides across Asia, killing more than 1,100 people in a week of destruction for the region and creating treacherous search and rescue operations for hundreds more still missing.
Millions of people are dealing with intense flooding after cyclone-fueled downpours battered parts of Indonesia, Thailand and Malaysia.
Sri Lanka was struck by a separate storm, leaving some areas submerged and creating the most difficult rescue operation the country has ever seen, according to President Anura Kumara Dissanayake.
“This is the first time the entire country has been struck by such a disaster,” he said in an address to the nation, Reuters reported. His comment suggested that the breadth of devastation has surpassed the damage caused in Sri Lanka by the Asian tsunami in 2004, which killed far more people.
Across Asia, the severe weather, which began last week,has so far claimed the lives of at least 604 people in Indonesia, 3